## 랜덤 포레스트
앙상블 학습의 대표주자
- 결정 트리를 랜덤하게 만들어 숲을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.
- 부트스트랩 샘플: 훈련 데이터에서 랜덤하게 샘플을 추출하여 생성, 중복 가능성이 있고 훈련 세트와 크기가 같다. 

In [1]:
import numpy as np
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

결과 값: 훈련세트에 과대적합

In [6]:
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


특성 중요도 확인해보기
- 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결과값 변경됨(234p 비교)

In [8]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


### OOB 샘플
out of bag: 부트스트랩 샘플에 포함되지 않고 남는 샘플
- 검증 세트 역할             

In [9]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리
부트스트랩 샘플을 사용하지 않는다.

교차 검증 점수 확인해보기
- 결과 값: 랜덤 포레스트와 비슷한 결과

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


특성 중요도 확인해보기

In [12]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블
- 경사 하강법 사용
- 과대적합에 강하다
- 성능이 좋지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느리다.

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1) # return_train_score=True : 훈련 세트의 점수도 함께 반환
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [15]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


랜덤 포레스트보다 일부 특성(당도)에 집중한다.

In [17]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


## 히스토그램 기반 그레이디언트 부스팅
입력 특성을 256개의 구간으로 나눠서 하나를 떼어 놓고, 누락된 값을 위해 사용한다.
- 입력에 누락된 특성이 있어도 따로 전처리할 필요가 없다.

In [18]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/home/leejieun/miniforge3/envs/ai/lib/python3.12/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


특성 중요도 결과 값: [특성 중요도, 평균, 표준편차]

In [19]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


테스트 세트에서 특성 중요도

In [22]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1) # n_repeats=10 : 랜덤하게 섞을 횟수
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [23]:
hgb.score(test_input, test_target)

0.8723076923076923

XGBoost 라이브러리를 사용하여 교차 검증 점수 구해보기

In [25]:
#conda install xgboost

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/leejieun/miniforge3/envs/ai

  added / updated specs:
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            gpu_0          12 KB  conda-forge
    cuda-version-12.4          |       h3060b56_3          21 KB  conda-forge
    libxgboost-2.0.3           |cuda120_h75debf4_1       163.5 MB  conda-forge
    nccl-2.20.5.1              |       h3a97aeb_0       104.0 MB  conda-forge
    py-xgboost-2.0.3           |cuda118_pyhedeaf28_1         129 KB  conda-forge
    xgboost-2.0.3   

In [26]:
from xgboost import XGBClassifier

In [27]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686
